In [301]:
import json
f = open("labels_v2.json")
label = json.load(f)
#ruled_based matcher
import spacy
nlp = spacy.load("/Users/luis/Documents/GitHub/2022Summer/AI-Oriented-Recommendation-System/PreprocessingData/spaCy/rule_model_7Categories")

In [302]:
pt = label.get("product_type")
st = label.get("sub_product_type")

In [303]:
st[0].keys()

dict_keys(['sandals', 'sneakers', 'heels', 'boots', 'shirts', 'sweaters', 'jackets/coats', 'activewear', 'shorts', 'skirts', 'pants', 'jeans', 'dresses', 'swimwear', 'jewellery', 'bags', 'glasses', 'headwear', 'cosmetics', 'skin care', 'beauty tools', 'hair care', 'nail care'])

In [100]:
#8 categories
shoes = set()
tops = set()
bottoms = set()
other_clothing = set()

beauty = set()
accessories = set()
homeware = set()
other = set()

In [101]:
shoes

set()

In [304]:
print(st[0].get('jeans'))

['Curvy Jeans', 'Jean Leggings', 'Mom Jeans', 'Dad Jeans', 'Straight Jeans', 'Bootcut Jeans', 'Flare Jeans', 'Culotte Jeans', 'Baggy Jeans', 'High-Rise Jeans', 'Cropped Jeans', 'Ripped Jeans', 'Skinny Jeans', 'Denim Jogger']


In [305]:
#method to compare two string similarities
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [306]:
similar("jacket", "jackets/coats")

0.631578947368421

In [307]:
from typing import List
def check_and_concat_punct(temp:List[str]) -> str:
    '''
    input: list, e.g. ['t', '-', 'shirt'], ['high', '-', 'rise', 'shoe']
    function: deal with the list with '-' punct
    output: "t-shirt" "high-rise shoe" "a-b-c color" 
    '''
    temp_string = ""
    if temp.__contains__('-') and len(temp) == 3:
        temp_string = ''.join(temp)
    elif temp.__contains__('-') and len(temp) > 3:
        
        pos = temp.index('-')
        #if there is wrong position of '-'
        if pos < 1:
            temp_string = ' '.join(temp)
            return temp_string
        
        puct_string = temp[pos - 1] + temp[pos] + temp[pos + 1]
        rest_string = ' '.join(temp[pos + 2:])
        temp_string = puct_string + ' '+ rest_string
    else:
        temp_string = ' '.join(temp)
    
    return temp_string 
    

In [309]:
'''
input: original string e.g. 'scandals', 'jackets/coats'ArithmeticError
funct: clean the string to the original words and put to list
output: ['scandal'], ['jacket', 'coats'] 
'''
def lemma_string(original_string:str) -> List:
    doc = nlp(original_string.lower())
    temp = []
    for token in doc:
        if token.text == '&' or token.text == 'and' or token.text == '/' or token.text == ',':
            break
        temp.append(token.lemma_)
    return temp


In [308]:
sub_cat_all_keys = [key for key in st[0].keys()]
sub_cat_shoes_keys = sub_cat_all_keys[0:4]
sub_cat_tops_keys= sub_cat_all_keys[4:7]
sub_cat_other_clothing_keys = sub_cat_all_keys[7:8] + sub_cat_all_keys[13:14]
sub_cat_bottoms_keys = sub_cat_all_keys[8:13]
sub_cat_acc_keys = sub_cat_all_keys[14: 19]
sub_cat_beauty_keys = sub_cat_all_keys[19: ]
print(sub_cat_shoes_keys)
print(sub_cat_tops_keys)
print(sub_cat_other_clothing_keys)
print(sub_cat_bottoms_keys)
print(sub_cat_acc_keys)
print(sub_cat_beauty_keys)



['sandals', 'sneakers', 'heels', 'boots']
['shirts', 'sweaters', 'jackets/coats']
['activewear', 'swimwear']
['shorts', 'skirts', 'pants', 'jeans', 'dresses']
['jewellery', 'bags', 'glasses', 'headwear', 'cosmetics']
['skin care', 'beauty tools', 'hair care', 'nail care']


In [310]:
#This map the product name to main_7categories type
#will be like {"slide" : 1} 1 is mapping to shoes
#from nltk.stem import PorterStemmer
#lemmatizer = PorterStemmer() #since this is not accurate as spacy.lemmanizer, don't use it.
all_main = {key for key in pt[0].keys()}
print("First: \n",  all_main)
#set
all_sub_main = set()
#dict { 'shoes': {(all shoe types whatever it is the 2nd or 3rd class label)} }
products_to_all = {}
#dict { 'shoes': 1, ''}
main_categories_map_to_num = {}
#dict { 'heels': 1, 't-shirt': } specific item maps to the number
specific_products_map_to_num = {}
pt_index = 0
for key in pt[0].keys():
    product_name_2nd_list = pt[0].get(key)
    pt_key = key
    pt_value_set = set()
    
    for item_string in product_name_2nd_list:
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)

        #update set
        all_sub_main.add(temp_string)
        #update the value in the pair[]
        pt_value_set.add(temp_string)
    
    #main_to_sub map
    products_to_all.update({pt_key:pt_value_set})
    #product_to_num map
    pt_index += 1
    main_categories_map_to_num.update({pt_key:pt_index})

main_categories_map_to_num.update({"other" : 8})
main_categories_map_to_num.update({"unknown" : 0})
#if key in [scandals, sneakers, heels, boots], or [boots, shirts, sweaters, jackets, coats]
#
print("Second: all main_categories\n", all_sub_main)

all_sub = set()
for key in st[0].keys():
    product_name_3rd_list = st[0].get(key) # get list of []
    for item_string in product_name_3rd_list: 
        #loop all item_string in order
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)
        all_sub.add(temp_string)
print("Third: all sub catrgories\n", all_sub)


First: 
 {'other_clothing', 'tops', 'bottoms', 'other', 'accessories', 'homeware', 'shoes', 'beauty'}
Second: all main_categories
 {'poncho', 'jumpsuit', 'sunglass', 'skate shoe', 'skin care', 'candle', 'flat shoe', 'coat', 'belt', 'backpack', 'collar', 'jacket', 'perfume', 'sweater', 'duvet cover', 'pyrrha', 'nail care', 'pant', 'skateboard tool', 'swimwear', 'bottle', 'cardigan', 'watch', 'heel cover', 'skirt', 'wallet', 'jean', 'card', 'drinkware', 'tank top', 'shawl', 'woodcarve', 'vest', 'waistcoat', 'tie', 'shoe lace', 'oxford', 'beauty tool', 'hair care', 'underwear', 'home decoration', 'canvas', 'umbrella', 'bath robe', 'glove', 'snowboard', 'jogger', 'kimono', 'heel', 'blouse', 'shower curtain', 'dress', 'phone case', 'bralette', 'wedge', 'soap', 'jewelry', 'shirt', 'slipper', 'wakesurfs', 'cosmetic', 'jersey', 'skateboard wax', 'sleepwear', 'boot', 'camisole', 'scarf', 'legge', 'sweatpant', 'short', 'air freshener', 'bath mat', 'sticker', 'artwork', 'sweatshirt', 'activewear'

In [311]:
'''
input: the list of all sub_categories, the key of the main_categories
function:put all sub_categories to the relative the main_categories set
output: none
'''
def update_products_to_all(string_list : List, main_key : str)->None:
    assert len(string_list) > 0
    assert products_to_all.get(main_key) is not None
    for item_string in string_list:
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp) 
        products_to_all.get(main_key).add(temp_string)


In [312]:
#add all sub categories to the main 8 categories.
for key in sub_cat_all_keys:#loop all keys in sub_main_categories, 
    if key in sub_cat_tops_keys:#if the key belongs to tops
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list, 'tops')
    elif key in sub_cat_shoes_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list, 'shoes')
    elif key in sub_cat_bottoms_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list, 'bottoms')
    elif key in sub_cat_other_clothing_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list, 'other_clothing')
    elif key in sub_cat_acc_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list, 'accessories')
    elif key in sub_cat_beauty_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list, 'beauty') 
    

In [313]:
products_to_all

{'shoes': {'ankle boot',
  'ankle strap heel',
  'boot',
  'canvas sneaker',
  'chelsea boot',
  'chunky sandal',
  'classic sneaker',
  'clog',
  'espadrille',
  'falt sandal',
  'flat shoe',
  'flip flop',
  'heel',
  'heel boot',
  'high top sneaker',
  'ice gripper boot',
  'jelly',
  'kitten heel',
  'lace-up boot',
  'mule',
  'oxford',
  'platform',
  'platform boot',
  'platform heel',
  'pump',
  'rain boot',
  'sandal',
  'skate shoe',
  'slide',
  'slip-on sneaker',
  'slipper',
  'sneaker',
  'snowshoe',
  'stilettos',
  'tall boot',
  'waterproof boot',
  'wedge',
  'wedge heel',
  'winter boot'},
 'tops': {'blazer',
  'blouse',
  'bomber jacket',
  'bra',
  'bralette',
  'button-down shirt',
  'button-up shirt',
  'camisole',
  'cardigan',
  'coat',
  'crop shirt',
  'cropped sweater',
  'denim jacket',
  'dress shirt',
  'hoodie',
  'jacket',
  'jersey',
  'kimono',
  'leather',
  'office shirt',
  'parka',
  'polo shirt',
  'poncho',
  'quilt',
  'raincoat',
  'shirt',


In [314]:
'''
Build the hashmap all products has mapped to the number of categories
'''
for key in products_to_all.keys():
    map_number_of_this_key = main_categories_map_to_num.get(key)
    assert products_to_all.get(key) is not None
    #include the main_categories to number;
    specific_products_map_to_num.update({key: map_number_of_this_key})
    for item in products_to_all.get(key):
        specific_products_map_to_num.update({item: map_number_of_this_key})
len(specific_products_map_to_num)


287

In [315]:
all_cat = all_sub.union(all_sub_main).union(all_main)

In [316]:
len(all_cat)

287

In [167]:
''' 
for item in all_cat:
    if item not in specific_products_map_to_num.keys():
        print(item)
'''

' \nfor item in all_cat:\n    if item not in specific_products_map_to_num.keys():\n        print(item)\n'

In [539]:
import pandas as pd
from string_grouper import match_strings, match_most_similar, \
    group_similar_strings, compute_pairwise_similarities
    
product_type = pd.read_csv("temp_not_finished.csv", usecols=['index', 'product_type'])
product_type.head(5)
product_type.head(-5)


,index,product_type
0,1,one piece
1,3,bikini top
2,4,bikini bottom
3,6,bikini top
4,7,bikini bottom
...,...,...
17963,22064,swimsuit
17964,22065,swimsuit
17965,22066,hat
17966,22067,bag


In [540]:
pre_defiened_labels = pd.Series(list(all_cat), name="pre_defined_label")
most_similar_matches = match_most_similar( pre_defiened_labels, product_type['product_type'],\
    min_similarity = 0.80, ignore_index=False, replace_na=False)
most_similar_matches

,most_similar_index,most_similar_pre_defined_label
0,8.0,one piece
1,257.0,bikini top
2,237.0,bikini bottom
3,257.0,bikini top
4,237.0,bikini bottom
...,...,...
17968,39.0,pant
17969,91.0,jewelry
17970,39.0,pant
17971,91.0,jewelry


In [543]:
most_similar_matches = pd.concat([product_type['index'],product_type['product_type'], most_similar_matches], axis=1)
most_similar_matches.to_csv("(GS)most_similar_string.csv")
most_similar_matches.shape


(17973, 6)

In [542]:
matches = match_strings(pre_defiened_labels, product_type['product_type'],\
    min_similarity = 0.60, ignore_index = False, replace_na=False)
matches.to_csv("(GS)match_strings.csv")
matches.shape

(10973, 5)

In [544]:
grouped_data = group_similar_strings(
    product_type['product_type'],
    min_similarity=0.60,
    ignore_index = False,
    replace_na = False
)
grouped_data.to_csv("(GS)group_similar_strings.csv")
grouped_data.shape

(17973, 2)

### Build 4 categories label dictionary


In [37]:
#accessory, other, 

In [545]:
all_cat.__contains__("perfume")

True

In [546]:
similar("acc", "accesories")

0.46153846153846156

In [547]:

df = pd.read_csv("temp_not_finished.csv")
df.insert(df.columns.get_loc('1st_class_label'), "match_most_similar_>80%_string", '', allow_duplicates=True)
df.insert(df.columns.get_loc('1st_class_label'), "match_string_>60%_similarity", '', allow_duplicates=True)
df.insert(df.columns.get_loc('1st_class_label'), "group_rep_product_type", '', allow_duplicates=True)


In [548]:
df.squeeze()

,Unnamed: 0,index,id,title,tags,body_html,product_type,match_most_similar_>80%_string,match_string_>60%_similarity,group_rep_product_type,1st_class_label,2nd_class_label,3rd_class_label,raw_text
0,0,1,6750552359068,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece,,,,0,0,0,Hues Underwire One Piece; d cup+ meta size cha...
1,1,3,7552752550106,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top,,,,0,0,0,Marble Malibu Set; crop meta size chart bound ...
2,2,4,5918371152028,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom,,,,0,0,0,Sophia Bottom; color patterned high waist meta...
3,3,6,5918535942300,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top,,,,0,0,0,Sonja Top; color stripes crop meta size chart ...
4,4,7,6613956362396,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom,,,,0,0,0,Shirley Tie Side; high waist meta size chart p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17968,17968,22069,2482003738706,Dree Crop Hi Rise / Cameo,"{""Citizens of Humanity"",Denim}","<meta charset=""utf-8"">\n<p class=""product-deta...",pant,,,,0,0,0,Dree Crop Hi Rise / Cameo; Citizens of Humanit...
17969,17969,22070,2408859238482,Rako Hoops / Sterling Silver,"{Earrings,""Fay Andrada""}","<meta charset=""utf-8"">\n<p><span>Lightweight h...",jewelry,,,,0,0,0,Rako Hoops / Sterling Silver; Earrings Fay And...
17970,17970,22071,1487071740013,Jamie Hi Rise Classic Fit / Plume,"{AGOLDE,Bottoms,Denim}","<meta charset=""utf-8"">\n<p><span data-v-19a2cf...",pant,,,,0,0,0,Jamie Hi Rise Classic Fit / Plume; AGOLDE Bott...
17971,17971,22072,147355598868,Tait Earrings in Brass and Malachite,"{Earrings,Quarry}",<p><span>The Tait Earrings are detailed with a...,jewelry,,,,0,0,0,Tait Earrings in Brass and Malachite; Earrings...


In [549]:
df.rename(columns={"Unnamed: 0":"current_index", "index": "original_index"}, inplace=True)
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags', 'body_html',
       'product_type', 'match_most_similar_>80%_string',
       'match_string_>60%_similarity', 'group_rep_product_type',
       '1st_class_label', '2nd_class_label', '3rd_class_label', 'raw_text'],
      dtype='object')

In [557]:
print(most_similar_matches.columns)
print(matches.columns)
print(grouped_data.columns)

Index(['index', 'product_type', 'index', 'product_type', 'most_similar_index',
       'most_similar_pre_defined_label'],
      dtype='object')
Index(['left_index', 'left_pre_defined_label', 'similarity',
       'right_product_type', 'right_index'],
      dtype='object')
Index(['group_rep_index', 'group_rep_product_type'], dtype='object')


In [558]:
most_similar_matches.columns.fillna()

Index(['index', 'product_type', 'index', 'product_type', 'most_similar_index',
       'most_similar_pre_defined_label'],
      dtype='object')

In [552]:
most_similar_matches.squeeze()

,index,product_type,index,product_type,most_similar_index,most_similar_pre_defined_label
0,1,one piece,1,one piece,8.0,one piece
1,3,bikini top,3,bikini top,257.0,bikini top
2,4,bikini bottom,4,bikini bottom,237.0,bikini bottom
3,6,bikini top,6,bikini top,257.0,bikini top
4,7,bikini bottom,7,bikini bottom,237.0,bikini bottom
...,...,...,...,...,...,...
17968,22069,pant,22069,pant,39.0,pant
17969,22070,jewelry,22070,jewelry,91.0,jewelry
17970,22071,pant,22071,pant,39.0,pant
17971,22072,jewelry,22072,jewelry,91.0,jewelry


In [555]:
import numpy as np
np.isnan(most_similar_matches.loc[17972, 'most_similar_index'].item())


True

In [588]:
i = most_similar_matches.at[1, 'index'].values[0]

df.loc[df['original_index'] == i, ['match_most_similar_>80%_string']]


,match_most_similar_>80%_string
1,one piece


In [589]:
#fill up the 80%
empty_cells = 0
for row in range(most_similar_matches.shape[0]):
    index = most_similar_matches.loc[row, 'index'].values[0]
    most_similar_pre_defined_label = most_similar_matches.loc[row, 'most_similar_pre_defined_label']
    most_similar_index = most_similar_matches.loc[row, ['most_similar_index']].item()
    
    if not np.isnan(most_similar_index):
        df.loc[df['original_index'] == index, ['match_most_similar_>80%_string']] = most_similar_pre_defined_label
    else:
        empty_cells += 1

In [591]:
df.squeeze()

,current_index,original_index,id,title,tags,body_html,product_type,match_most_similar_>80%_string,match_string_>60%_similarity,group_rep_product_type,1st_class_label,2nd_class_label,3rd_class_label,raw_text
0,0,1,6750552359068,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece,one piece,,,0,0,0,Hues Underwire One Piece; d cup+ meta size cha...
1,1,3,7552752550106,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top,bikini top,,,0,0,0,Marble Malibu Set; crop meta size chart bound ...
2,2,4,5918371152028,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom,bikini bottom,,,0,0,0,Sophia Bottom; color patterned high waist meta...
3,3,6,5918535942300,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top,bikini top,,,0,0,0,Sonja Top; color stripes crop meta size chart ...
4,4,7,6613956362396,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom,bikini bottom,,,0,0,0,Shirley Tie Side; high waist meta size chart p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17968,17968,22069,2482003738706,Dree Crop Hi Rise / Cameo,"{""Citizens of Humanity"",Denim}","<meta charset=""utf-8"">\n<p class=""product-deta...",pant,pant,,,0,0,0,Dree Crop Hi Rise / Cameo; Citizens of Humanit...
17969,17969,22070,2408859238482,Rako Hoops / Sterling Silver,"{Earrings,""Fay Andrada""}","<meta charset=""utf-8"">\n<p><span>Lightweight h...",jewelry,jewelry,,,0,0,0,Rako Hoops / Sterling Silver; Earrings Fay And...
17970,17970,22071,1487071740013,Jamie Hi Rise Classic Fit / Plume,"{AGOLDE,Bottoms,Denim}","<meta charset=""utf-8"">\n<p><span data-v-19a2cf...",pant,pant,,,0,0,0,Jamie Hi Rise Classic Fit / Plume; AGOLDE Bott...
17971,17971,22072,147355598868,Tait Earrings in Brass and Malachite,"{Earrings,Quarry}",<p><span>The Tait Earrings are detailed with a...,jewelry,jewelry,,,0,0,0,Tait Earrings in Brass and Malachite; Earrings...


In [590]:
empty_cells

10463

In [592]:
#fill up 60 %
index_similarity_map = {}
index_to_pre_map = {}
for row in range(matches.shape[0]):
    current_index = matches.loc[row, 'right_index']
    label = matches.loc[row, 'left_pre_defined_label']
    similarity = matches.loc[row, 'similarity']
    
    if index_similarity_map.get(current_index) == None:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    elif index_similarity_map.get(current_index) >= similarity:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    else: # similarity < then current, ignore
        continue


In [593]:
for key in index_to_pre_map.keys():
    df.loc[df['current_index'] == key, ['match_string_>60%_similarity']] = index_to_pre_map.get(key)

In [594]:
#fill up group representative of the label 
for row in range(grouped_data.shape[0]):
    df.loc[row, 'group_rep_product_type'] = grouped_data.loc[row, 'group_rep_product_type']

In [596]:
#map most_similar_label to number_of_categories using specific_product_to_num_map
for row in range(df.shape[0]):
    
    key = df.loc[row, ['match_most_similar_>80%_string']].item()
    if key != None and specific_products_map_to_num.get(key) != None:
        df.loc[row, ['1st_class_label']] = specific_products_map_to_num.get(key)
    else:
        df.loc[row, ['1st_class_label']] = 0
        

In [597]:
df.squeeze()

,current_index,original_index,id,title,tags,body_html,product_type,match_most_similar_>80%_string,match_string_>60%_similarity,group_rep_product_type,1st_class_label,2nd_class_label,3rd_class_label,raw_text
0,0,1,6750552359068,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece,one piece,one piece,one piece,4,0,0,Hues Underwire One Piece; d cup+ meta size cha...
1,1,3,7552752550106,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top,bikini top,bikini top,black string bikini bottoms,4,0,0,Marble Malibu Set; crop meta size chart bound ...
2,2,4,5918371152028,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom,bikini bottom,bikini bottom,black string bikini bottoms,4,0,0,Sophia Bottom; color patterned high waist meta...
3,3,6,5918535942300,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top,bikini top,bikini top,black string bikini bottoms,4,0,0,Sonja Top; color stripes crop meta size chart ...
4,4,7,6613956362396,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom,bikini bottom,bikini bottom,black string bikini bottoms,4,0,0,Shirley Tie Side; high waist meta size chart p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17968,17968,22069,2482003738706,Dree Crop Hi Rise / Cameo,"{""Citizens of Humanity"",Denim}","<meta charset=""utf-8"">\n<p class=""product-deta...",pant,pant,pant,pants,3,0,0,Dree Crop Hi Rise / Cameo; Citizens of Humanit...
17969,17969,22070,2408859238482,Rako Hoops / Sterling Silver,"{Earrings,""Fay Andrada""}","<meta charset=""utf-8"">\n<p><span>Lightweight h...",jewelry,jewelry,jewelry,jewelry,6,0,0,Rako Hoops / Sterling Silver; Earrings Fay And...
17970,17970,22071,1487071740013,Jamie Hi Rise Classic Fit / Plume,"{AGOLDE,Bottoms,Denim}","<meta charset=""utf-8"">\n<p><span data-v-19a2cf...",pant,pant,pant,pants,3,0,0,Jamie Hi Rise Classic Fit / Plume; AGOLDE Bott...
17971,17971,22072,147355598868,Tait Earrings in Brass and Malachite,"{Earrings,Quarry}",<p><span>The Tait Earrings are detailed with a...,jewelry,jewelry,jewelry,jewelry,6,0,0,Tait Earrings in Brass and Malachite; Earrings...


In [598]:
df.to_csv('(temp)first_class_label_done.csv')

In [599]:
unknown = 0
total = df.shape[0]
shoes = 0
other_clothing = 0
tops = 0
bottoms = 0
beauty = 0
home = 0
acc = 0
other = 0
for row in range(df.shape[0]):
    n = df.loc[row, ['1st_class_label']].item()
    if n == 0:
        unknown += 1
    elif n == 1:
        shoes += 1
    elif n == 2:
        tops += 1
    elif n == 3:
        bottoms += 1
    elif n == 4:
        other_clothing += 1
    elif n == 5:
        beauty += 1
    elif n == 6:
        acc += 1
    elif n == 7:
        home += 1
    elif n == 8:
        other += 1

In [600]:
totalnum = total

In [604]:
print("unknown", unknown, "\tratio of all", unknown/total)
total = totalnum - unknown
print("\nshoes", shoes,"\tratio", shoes/total)
print("tops", tops, "\tratio", tops/total)
print("bottoms", bottoms, "\tratio", bottoms/total)
print("other_clothing", other_clothing,"\tratio", other_clothing/total)
print("beauty", beauty, "\tratio", beauty/total)
print("accessories", acc, "\tratio", acc/total)
print("homeware", home, "\tratio", home/total)
print("other", other, "\tratio", other/total)


unknown 6985 	ratio of all 0.6356934838005096

shoes 830 	ratio 0.07553694939934474
tops 2740 	ratio 0.24936294139060794
bottoms 1717 	ratio 0.1562613760465963
other_clothing 1218 	ratio 0.11084819803421915
beauty 731 	ratio 0.06652712049508555
accessories 3321 	ratio 0.30223880597014924
homeware 349 	ratio 0.03176192209683291
other 82 	ratio 0.007462686567164179
